In [1]:
!jupyter nbconvert --to script repair.ipynb
from repair import *

[NbConvertApp] Converting notebook repair.ipynb to script
[NbConvertApp] Writing 11316 bytes to repair.py


In [2]:
def side_by_side(lines1, lines2, gap=4):
    max_len = max(len(lines1), len(lines2))
    lines1 += [''] * (max_len - len(lines1))
    lines2 += [''] * (max_len - len(lines2))
    return [f"{a:<60}{' ' * gap}{b}" for a, b in zip(lines1, lines2)]
    
def side_by_side_dicts(dict1, dict2, title1="Before", title2="After", key_label="Key"):
    all_keys = sorted(set(dict1) | set(dict2))
    lines = [f"{key_label:<6} {title1:<30}    {title2}"]
    lines.append("-" * 80)
    for key in all_keys:
        val1 = str(dict1.get(key, ""))
        val2 = str(dict2.get(key, ""))
        print(f"{str(key):<6} {val1:<30}    {val2}")

def compare_structures(obj1, obj2):
    eq = True
    if isinstance(obj1, list) and isinstance(obj2, list):
        if len(obj1) != len(obj2):
            print(f"List lengths differ: {len(obj1)} vs {len(obj2)}")
            eq = False
        for i, (a, b) in enumerate(zip(obj1, obj2)):
            if a != b:
                eq = False
                print(f"List elements differ at index {i}:\n  Left:  {a}\n  Right: {b}")
        if len(obj1) != len(obj2):
            extra = obj1[len(obj2):] if len(obj1) > len(obj2) else obj2[len(obj1):]
            eq = False
            print(f"Extra elements in {'left' if len(obj1) > len(obj2) else 'right'} list: {extra}")

    elif isinstance(obj1, dict) and isinstance(obj2, dict):
        keys1 = set(obj1.keys())
        keys2 = set(obj2.keys())
        all_keys = keys1 | keys2
        for key in all_keys:
            a = obj1.get(key, None)
            b = obj2.get(key, None)
            if a != b:
                eq = False
                print(f"Dict entries differ for key {key}:\n  Left:  {a}\n  Right: {b}")
    else:
        eq = False
        print("Unsupported types for comparison. Use only lists or dicts.")
    if eq:
        print("OK")

from pprint import pprint
import copy



def debug(string, k_tuple, new_symbol, ab, aa, sb, sa, pqb, pqa):
    sequence, active_k_tuples = construct_active_k_tuples_and_sequence(string_to_symbol_list(string)[0], 2)
    priority_queue = construct_priority_queue(active_k_tuples)

    print("\nactive_k_tuples_before = \\", sep="")
    pprint(active_k_tuples)
    print("\nsequence_before = \\", sep="")
    pprint(sequence)
    print("\npriority_queue_before = \\", sep="")
    pprint(priority_queue)
    
    
    s_active_before = KTupleInfo.get_print_lines(active_k_tuples)
    s_seq_before = SequenceElement.get_print_lines(sequence)
    queue_before = copy.deepcopy(priority_queue)
    
    replace_active_k_tuple(priority_queue, active_k_tuples, sequence, k_tuple, new_symbol, 2)

    s_active_after = KTupleInfo.get_print_lines(active_k_tuples)
    s_seq_after = SequenceElement.get_print_lines(sequence)
    s_queue_after = str(priority_queue)

    print("\nactive_k_tuples_after = \\", sep="")
    pprint(active_k_tuples)
    print("\nsequence_after = \\", sep="")
    pprint(sequence)
    print("\npriority_queue_after = \\", sep="")
    pprint(priority_queue)

    
    print("active_k_tuples")
    compare_structures(active_k_tuples, aa)
    print("sequence ")
    compare_structures(sequence, sa)
    print("priority_queue")
    compare_structures(priority_queue, pqa)

    print("\n" + "-" * 100)
    print("Active K-Tuples (before vs after)")
    for line in side_by_side(s_active_before, s_active_after):
        print(line)

    print("\n" + "-" * 100)
    print("Sequence (before vs after)")
    for line in side_by_side(s_seq_before, s_seq_after):
        print(line)
    print("-" * 100)

    print("\n" + "-" * 100)
    print("Priority queue (before vs after)")
    side_by_side_dicts(queue_before, priority_queue)
    print("-" * 100)

In [6]:
active_k_tuples_before = \
{(0, 1): KTupleInfo(count=2, last=3), (1, 0): KTupleInfo(count=1, last=2)}

sequence_before = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=-1, pos=1, next_k_tuple=3),
 SequenceElement(symbol=0, prev_k_tuple=-1, pos=2, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=1, pos=3, next_k_tuple=-1)]

priority_queue_before = \
{1: {(1, 0)}, 2: {(0, 1)}}

active_k_tuples_after = \
{(0, 3): KTupleInfo(count=1, last=2), (3, 1): KTupleInfo(count=1, last=3)}

sequence_after = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=-1, prev_k_tuple=0, pos=1, next_k_tuple=2),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=2, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=-1, pos=3, next_k_tuple=-1)]

priority_queue_after = \
{1: {(3, 1), (0, 3)}, 2: set()}

debug("0101", (1,0), 3, active_k_tuples_before, active_k_tuples_after, sequence_before, sequence_after, priority_queue_before, priority_queue_after)
print("-------------------------------------------------------------------------------------------")
print("                                          OK                                               ")
print("-------------------------------------------------------------------------------------------")



active_k_tuples_before = \
{(0, 1): KTupleInfo(count=2, last=3), (1, 0): KTupleInfo(count=1, last=2)}

sequence_before = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=-1, pos=1, next_k_tuple=3),
 SequenceElement(symbol=0, prev_k_tuple=-1, pos=2, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=1, pos=3, next_k_tuple=-1)]

priority_queue_before = \
{1: {(1, 0)}, 2: {(0, 1)}}

active_k_tuples_after = \
{(0, 3): KTupleInfo(count=1, last=2), (3, 1): KTupleInfo(count=1, last=3)}

sequence_after = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=-1, prev_k_tuple=0, pos=1, next_k_tuple=2),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=2, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=-1, pos=3, next_k_tuple=-1)]

priority_queue_after = \
{1: {(3, 1), (0, 3)}, 2: set()}
active_k_tuples
OK
sequence 
OK
priority_queue
OK

------------------------------------------

In [7]:
#A and D - no old pairs elsewhere, no new pairs
active_k_tuples_before = \
{(0, 1): KTupleInfo(count=1, last=1),
 (1, 2): KTupleInfo(count=1, last=2),
 (2, 3): KTupleInfo(count=1, last=3)}

sequence_before = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=-1, pos=1, next_k_tuple=-1),
 SequenceElement(symbol=2, prev_k_tuple=-1, pos=2, next_k_tuple=-1),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=3, next_k_tuple=-1)]

priority_queue_before = \
{1: {(0, 1), (1, 2), (2, 3)}}

active_k_tuples_after = \
{(0, 4): KTupleInfo(count=1, last=2), (4, 3): KTupleInfo(count=1, last=3)}

sequence_after = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=-1, prev_k_tuple=0, pos=1, next_k_tuple=2),
 SequenceElement(symbol=4, prev_k_tuple=-1, pos=2, next_k_tuple=-1),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=3, next_k_tuple=-1)]

priority_queue_after = \
{1: {(4, 3), (0, 4)}}

debug("0123", (1,2), 4, active_k_tuples_before, active_k_tuples_after, sequence_before, sequence_after, priority_queue_before, priority_queue_after)
print("-------------------------------------------------------------------------------------------")
print("                                          OK                                               ")
print("-------------------------------------------------------------------------------------------")


active_k_tuples_before = \
{(0, 1): KTupleInfo(count=1, last=1),
 (1, 2): KTupleInfo(count=1, last=2),
 (2, 3): KTupleInfo(count=1, last=3)}

sequence_before = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=-1, pos=1, next_k_tuple=-1),
 SequenceElement(symbol=2, prev_k_tuple=-1, pos=2, next_k_tuple=-1),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=3, next_k_tuple=-1)]

priority_queue_before = \
{1: {(0, 1), (1, 2), (2, 3)}}

active_k_tuples_after = \
{(0, 4): KTupleInfo(count=1, last=2), (4, 3): KTupleInfo(count=1, last=3)}

sequence_after = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=-1, prev_k_tuple=0, pos=1, next_k_tuple=2),
 SequenceElement(symbol=4, prev_k_tuple=-1, pos=2, next_k_tuple=-1),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=3, next_k_tuple=-1)]

priority_queue_after = \
{1: {(4, 3), (0, 4)}}
active_k_tuples
OK
sequence 
OK
priority_queue
OK

--------

In [8]:
#A and D - old pairs to left and right
active_k_tuples_before = \
{(0, 1): KTupleInfo(count=3, last=9),
 (1, 0): KTupleInfo(count=1, last=8),
 (1, 2): KTupleInfo(count=2, last=10),
 (2, 3): KTupleInfo(count=3, last=11),
 (3, 0): KTupleInfo(count=1, last=6),
 (3, 2): KTupleInfo(count=1, last=4)}

sequence_before = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=-1, pos=1, next_k_tuple=7),
 SequenceElement(symbol=2, prev_k_tuple=-1, pos=2, next_k_tuple=10),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=3, next_k_tuple=5),
 SequenceElement(symbol=2, prev_k_tuple=-1, pos=4, next_k_tuple=-1),
 SequenceElement(symbol=3, prev_k_tuple=3, pos=5, next_k_tuple=11),
 SequenceElement(symbol=0, prev_k_tuple=-1, pos=6, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=1, pos=7, next_k_tuple=9),
 SequenceElement(symbol=0, prev_k_tuple=-1, pos=8, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=7, pos=9, next_k_tuple=-1),
 SequenceElement(symbol=2, prev_k_tuple=2, pos=10, next_k_tuple=-1),
 SequenceElement(symbol=3, prev_k_tuple=5, pos=11, next_k_tuple=-1)]

priority_queue_before = \
{1: {(1, 0), (3, 2), (3, 0)}, 2: {(1, 2)}, 3: {(0, 1), (2, 3)}}

active_k_tuples_after = \
{(0, 1): KTupleInfo(count=2, last=9),
 (1, 0): KTupleInfo(count=1, last=8),
 (1, 2): KTupleInfo(count=2, last=10),
 (2, 3): KTupleInfo(count=2, last=11),
 (2, 5): KTupleInfo(count=1, last=6),
 (3, 2): KTupleInfo(count=1, last=4),
 (5, 1): KTupleInfo(count=1, last=7)}

sequence_after = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=-1, pos=1, next_k_tuple=9),
 SequenceElement(symbol=2, prev_k_tuple=-1, pos=2, next_k_tuple=10),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=3, next_k_tuple=11),
 SequenceElement(symbol=2, prev_k_tuple=-1, pos=4, next_k_tuple=-1),
 SequenceElement(symbol=-1, prev_k_tuple=4, pos=5, next_k_tuple=6),
 SequenceElement(symbol=5, prev_k_tuple=-1, pos=6, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=-1, pos=7, next_k_tuple=-1),
 SequenceElement(symbol=0, prev_k_tuple=-1, pos=8, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=1, pos=9, next_k_tuple=-1),
 SequenceElement(symbol=2, prev_k_tuple=2, pos=10, next_k_tuple=-1),
 SequenceElement(symbol=3, prev_k_tuple=3, pos=11, next_k_tuple=-1)]

priority_queue_after = \
{1: {(5, 1), (1, 0), (3, 2), (2, 5)}, 2: {(0, 1), (1, 2), (2, 3)}, 3: set()}

debug("CDABABCDCDAB", (3,0), 5, active_k_tuples_before, active_k_tuples_after, sequence_before, sequence_after, priority_queue_before, priority_queue_after)
print("-------------------------------------------------------------------------------------------")
print("                                          OK                                               ")
print("-------------------------------------------------------------------------------------------")


active_k_tuples_before = \
{(0, 1): KTupleInfo(count=3, last=9),
 (1, 0): KTupleInfo(count=1, last=8),
 (1, 2): KTupleInfo(count=2, last=10),
 (2, 3): KTupleInfo(count=3, last=11),
 (3, 0): KTupleInfo(count=1, last=6),
 (3, 2): KTupleInfo(count=1, last=4)}

sequence_before = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=-1, pos=1, next_k_tuple=7),
 SequenceElement(symbol=2, prev_k_tuple=-1, pos=2, next_k_tuple=10),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=3, next_k_tuple=5),
 SequenceElement(symbol=2, prev_k_tuple=-1, pos=4, next_k_tuple=-1),
 SequenceElement(symbol=3, prev_k_tuple=3, pos=5, next_k_tuple=11),
 SequenceElement(symbol=0, prev_k_tuple=-1, pos=6, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=1, pos=7, next_k_tuple=9),
 SequenceElement(symbol=0, prev_k_tuple=-1, pos=8, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=7, pos=9, next_k_tuple=-1),
 SequenceElement(symbol=2, prev_k_tup

In [10]:
#first old pairs replaced 
active_k_tuples_before = \
{(0, 1): KTupleInfo(count=2, last=7),
 (1, 2): KTupleInfo(count=1, last=2),
 (2, 3): KTupleInfo(count=2, last=5),
 (3, 0): KTupleInfo(count=1, last=6),
 (3, 2): KTupleInfo(count=1, last=4)}

sequence_before = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=-1, pos=1, next_k_tuple=7),
 SequenceElement(symbol=2, prev_k_tuple=-1, pos=2, next_k_tuple=-1),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=3, next_k_tuple=5),
 SequenceElement(symbol=2, prev_k_tuple=-1, pos=4, next_k_tuple=-1),
 SequenceElement(symbol=3, prev_k_tuple=3, pos=5, next_k_tuple=-1),
 SequenceElement(symbol=0, prev_k_tuple=-1, pos=6, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=1, pos=7, next_k_tuple=-1)]

priority_queue_before = \
{1: {(3, 2), (1, 2), (3, 0)}, 2: {(0, 1), (2, 3)}}

active_k_tuples_after = \
{(0, 1): KTupleInfo(count=1, last=1),
 (1, 2): KTupleInfo(count=1, last=2),
 (2, 3): KTupleInfo(count=1, last=3),
 (2, 5): KTupleInfo(count=1, last=6),
 (3, 2): KTupleInfo(count=1, last=4),
 (5, 1): KTupleInfo(count=1, last=7)}

sequence_after = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=-1, pos=1, next_k_tuple=-1),
 SequenceElement(symbol=2, prev_k_tuple=-1, pos=2, next_k_tuple=-1),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=3, next_k_tuple=-1),
 SequenceElement(symbol=2, prev_k_tuple=-1, pos=4, next_k_tuple=-1),
 SequenceElement(symbol=-1, prev_k_tuple=4, pos=5, next_k_tuple=6),
 SequenceElement(symbol=5, prev_k_tuple=-1, pos=6, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=-1, pos=7, next_k_tuple=-1)]

priority_queue_after = \
{1: {(0, 1), (1, 2), (5, 1), (2, 3), (3, 2), (2, 5)}, 2: set()}

debug("23010123", (3,0), 5, active_k_tuples_before, active_k_tuples_after, sequence_before, sequence_after, priority_queue_before, priority_queue_after)
print("-------------------------------------------------------------------------------------------")
print("                                          OK                                               ")
print("-------------------------------------------------------------------------------------------")


active_k_tuples_before = \
{(0, 1): KTupleInfo(count=2, last=7),
 (1, 2): KTupleInfo(count=1, last=2),
 (2, 3): KTupleInfo(count=2, last=5),
 (3, 0): KTupleInfo(count=1, last=6),
 (3, 2): KTupleInfo(count=1, last=4)}

sequence_before = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=-1, pos=1, next_k_tuple=7),
 SequenceElement(symbol=2, prev_k_tuple=-1, pos=2, next_k_tuple=-1),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=3, next_k_tuple=5),
 SequenceElement(symbol=2, prev_k_tuple=-1, pos=4, next_k_tuple=-1),
 SequenceElement(symbol=3, prev_k_tuple=3, pos=5, next_k_tuple=-1),
 SequenceElement(symbol=0, prev_k_tuple=-1, pos=6, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=1, pos=7, next_k_tuple=-1)]

priority_queue_before = \
{1: {(3, 2), (1, 2), (3, 0)}, 2: {(0, 1), (2, 3)}}

active_k_tuples_after = \
{(0, 1): KTupleInfo(count=1, last=1),
 (1, 2): KTupleInfo(count=1, last=2),
 (2, 3): KTupleInfo(count=1, las

In [12]:
# ABAB - replacing BA
active_k_tuples_before = \
{(0, 1): KTupleInfo(count=2, last=3), (1, 0): KTupleInfo(count=1, last=2)}

sequence_before = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=-1, pos=1, next_k_tuple=3),
 SequenceElement(symbol=0, prev_k_tuple=-1, pos=2, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=1, pos=3, next_k_tuple=-1)]

priority_queue_before = \
{1: {(1, 0)}, 2: {(0, 1)}}

active_k_tuples_after = \
{(2, 2): KTupleInfo(count=1, last=3)}

sequence_after = \
[SequenceElement(symbol=-1, prev_k_tuple=-1, pos=0, next_k_tuple=1),
 SequenceElement(symbol=2, prev_k_tuple=-1, pos=1, next_k_tuple=-1),
 SequenceElement(symbol=-1, prev_k_tuple=1, pos=2, next_k_tuple=3),
 SequenceElement(symbol=2, prev_k_tuple=-1, pos=3, next_k_tuple=-1)]

priority_queue_after = \
{1: {(2, 2)}, 2: set()}
debug("0101", (0,1), 2, active_k_tuples_before, active_k_tuples_after, sequence_before, sequence_after, priority_queue_before, priority_queue_after)
print("-------------------------------------------------------------------------------------------")
print("                                          OK                                               ")
print("-------------------------------------------------------------------------------------------")
# AAA - replacing AA



active_k_tuples_before = \
{(0, 1): KTupleInfo(count=2, last=3), (1, 0): KTupleInfo(count=1, last=2)}

sequence_before = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=-1, pos=1, next_k_tuple=3),
 SequenceElement(symbol=0, prev_k_tuple=-1, pos=2, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=1, pos=3, next_k_tuple=-1)]

priority_queue_before = \
{1: {(1, 0)}, 2: {(0, 1)}}

active_k_tuples_after = \
{(2, 2): KTupleInfo(count=1, last=3)}

sequence_after = \
[SequenceElement(symbol=-1, prev_k_tuple=-1, pos=0, next_k_tuple=1),
 SequenceElement(symbol=2, prev_k_tuple=-1, pos=1, next_k_tuple=-1),
 SequenceElement(symbol=-1, prev_k_tuple=1, pos=2, next_k_tuple=3),
 SequenceElement(symbol=2, prev_k_tuple=-1, pos=3, next_k_tuple=-1)]

priority_queue_after = \
{1: {(2, 2)}, 2: set()}
active_k_tuples
OK
sequence 
OK
priority_queue
OK

---------------------------------------------------------------------------------------

In [14]:
active_k_tuples_before = \
{(0, 1): KTupleInfo(count=3, last=5), (1, 0): KTupleInfo(count=2, last=4)}

sequence_before = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=-1, pos=1, next_k_tuple=3),
 SequenceElement(symbol=0, prev_k_tuple=-1, pos=2, next_k_tuple=4),
 SequenceElement(symbol=1, prev_k_tuple=1, pos=3, next_k_tuple=5),
 SequenceElement(symbol=0, prev_k_tuple=2, pos=4, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=3, pos=5, next_k_tuple=-1)]

priority_queue_before = \
{1: set(), 2: {(1, 0)}, 3: {(0, 1)}}

active_k_tuples_after = \
{(3, 3): KTupleInfo(count=2, last=5)}

sequence_after = \
[SequenceElement(symbol=-1, prev_k_tuple=-1, pos=0, next_k_tuple=1),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=1, next_k_tuple=-1),
 SequenceElement(symbol=-1, prev_k_tuple=1, pos=2, next_k_tuple=3),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=3, next_k_tuple=5),
 SequenceElement(symbol=-1, prev_k_tuple=3, pos=4, next_k_tuple=5),
 SequenceElement(symbol=3, prev_k_tuple=3, pos=5, next_k_tuple=-1)]

priority_queue_after = \
{1: set(), 2: {(3, 3)}, 3: set()}

debug("010101", (0,1), 3, active_k_tuples_before, active_k_tuples_after, sequence_before, sequence_after, priority_queue_before, priority_queue_after)
print("-------------------------------------------------------------------------------------------")
print("                                          OK                                               ")
print("-------------------------------------------------------------------------------------------")


active_k_tuples_before = \
{(0, 1): KTupleInfo(count=3, last=5), (1, 0): KTupleInfo(count=2, last=4)}

sequence_before = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=-1, pos=1, next_k_tuple=3),
 SequenceElement(symbol=0, prev_k_tuple=-1, pos=2, next_k_tuple=4),
 SequenceElement(symbol=1, prev_k_tuple=1, pos=3, next_k_tuple=5),
 SequenceElement(symbol=0, prev_k_tuple=2, pos=4, next_k_tuple=-1),
 SequenceElement(symbol=1, prev_k_tuple=3, pos=5, next_k_tuple=-1)]

priority_queue_before = \
{1: set(), 2: {(1, 0)}, 3: {(0, 1)}}

active_k_tuples_after = \
{(3, 3): KTupleInfo(count=2, last=5)}

sequence_after = \
[SequenceElement(symbol=-1, prev_k_tuple=-1, pos=0, next_k_tuple=1),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=1, next_k_tuple=-1),
 SequenceElement(symbol=-1, prev_k_tuple=1, pos=2, next_k_tuple=3),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=3, next_k_tuple=5),
 SequenceElement(symbol=-1, prev_k_tuple=3, p

In [16]:
active_k_tuples_before = \
{(0, 0): KTupleInfo(count=2, last=2)}

sequence_before = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=0, prev_k_tuple=-1, pos=1, next_k_tuple=2),
 SequenceElement(symbol=0, prev_k_tuple=1, pos=2, next_k_tuple=-1)]

priority_queue_before = \
{1: set(), 2: {(0, 0)}}

active_k_tuples_after = \
{(0, 3): KTupleInfo(count=1, last=2)}

sequence_after = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=-1, prev_k_tuple=0, pos=1, next_k_tuple=2),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=2, next_k_tuple=-1)]

priority_queue_after = \
{1: {(0, 3)}, 2: set()}

debug("000", (0,0), 3, active_k_tuples_before, active_k_tuples_after, sequence_before, sequence_after, priority_queue_before, priority_queue_after)



active_k_tuples_before = \
{(0, 0): KTupleInfo(count=2, last=2)}

sequence_before = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=0, prev_k_tuple=-1, pos=1, next_k_tuple=2),
 SequenceElement(symbol=0, prev_k_tuple=1, pos=2, next_k_tuple=-1)]

priority_queue_before = \
{1: set(), 2: {(0, 0)}}

active_k_tuples_after = \
{(0, 3): KTupleInfo(count=1, last=2)}

sequence_after = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=-1, prev_k_tuple=0, pos=1, next_k_tuple=2),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=2, next_k_tuple=-1)]

priority_queue_after = \
{1: {(0, 3)}, 2: set()}
active_k_tuples
OK
sequence 
OK
priority_queue
OK

----------------------------------------------------------------------------------------------------
Active K-Tuples (before vs after)
Key    Count  Last                                              Key    Count  Last  
--------------------------------             

In [18]:
active_k_tuples_before = \
{(0, 0): KTupleInfo(count=3, last=3)}

sequence_before = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=0, prev_k_tuple=-1, pos=1, next_k_tuple=2),
 SequenceElement(symbol=0, prev_k_tuple=1, pos=2, next_k_tuple=3),
 SequenceElement(symbol=0, prev_k_tuple=2, pos=3, next_k_tuple=-1)]

priority_queue_before = \
{1: set(), 2: set(), 3: {(0, 0)}}

active_k_tuples_after = \
{(3, 3): KTupleInfo(count=1, last=3)}

sequence_after = \
[SequenceElement(symbol=-1, prev_k_tuple=-1, pos=0, next_k_tuple=1),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=1, next_k_tuple=-1),
 SequenceElement(symbol=-1, prev_k_tuple=1, pos=2, next_k_tuple=3),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=3, next_k_tuple=-1)]

priority_queue_after = \
{1: {(3, 3)}, 2: set(), 3: set()}

debug("0000", (0,0), 3, active_k_tuples_before, active_k_tuples_after, sequence_before, sequence_after, priority_queue_before, priority_queue_after)
print("-------------------------------------------------------------------------------------------")
print("                                          OK                                               ")
print("-------------------------------------------------------------------------------------------")
print("important! in this case we create a new active_k_tuple which is then remvoed!!!")


active_k_tuples_before = \
{(0, 0): KTupleInfo(count=3, last=3)}

sequence_before = \
[SequenceElement(symbol=0, prev_k_tuple=-1, pos=0, next_k_tuple=-1),
 SequenceElement(symbol=0, prev_k_tuple=-1, pos=1, next_k_tuple=2),
 SequenceElement(symbol=0, prev_k_tuple=1, pos=2, next_k_tuple=3),
 SequenceElement(symbol=0, prev_k_tuple=2, pos=3, next_k_tuple=-1)]

priority_queue_before = \
{1: set(), 2: set(), 3: {(0, 0)}}

active_k_tuples_after = \
{(3, 3): KTupleInfo(count=1, last=3)}

sequence_after = \
[SequenceElement(symbol=-1, prev_k_tuple=-1, pos=0, next_k_tuple=1),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=1, next_k_tuple=-1),
 SequenceElement(symbol=-1, prev_k_tuple=1, pos=2, next_k_tuple=3),
 SequenceElement(symbol=3, prev_k_tuple=-1, pos=3, next_k_tuple=-1)]

priority_queue_after = \
{1: {(3, 3)}, 2: set(), 3: set()}
active_k_tuples
OK
sequence 
OK
priority_queue
OK

----------------------------------------------------------------------------------------------------
Active K

In [13]:
s, a = construct_active_k_tuples_and_sequence(string_to_symbol_list("0101")[0], 2)
for x in SequenceElement.get_print_lines(s): print(x)
for x in KTupleInfo.get_print_lines(a): print(x)

Index  Symbol     Pos    PrevKTuple   NextKTuple  
--------------------------------------------------
0      0          0      -1           -1          
1      1          1      -1           3           
2      0          2      -1           -1          
3      1          3      1            -1          
Key    Count  Last   PosInQueue  
--------------------------------
(0, 1) 2      3      -1          
(1, 0) 1      2      -1          


In [14]:
s, a = construct_active_k_tuples_and_sequence(string_to_symbol_list("000")[0], 2)
for x in SequenceElement.get_print_lines(s): print(x)
for x in KTupleInfo.get_print_lines(a): print(x)

Index  Symbol     Pos    PrevKTuple   NextKTuple  
--------------------------------------------------
0      0          0      -1           -1          
1      0          1      -1           2           
2      0          2      1            -1          
Key    Count  Last   PosInQueue  
--------------------------------
(0, 0) 2      2      -1          


In [67]:
s, a = construct_active_k_tuples_and_sequence(string_to_symbol_list("012012")[0], 2)
SequenceElement.print_list(s)
KTupleInfo.print_dict(a)

Index  Symbol     Pos    PrevKTuple   NextKTuple  
--------------------------------------------------
0      0          0      -1           -1          
1      1          1      -1           4           
2      2          2      -1           5           
3      0          3      -1           -1          
4      1          4      1            -1          
5      2          5      2            -1          
Key    Count  Last   PosInQueue  
--------------------------------
(0, 1) 2      4      -1          
(1, 2) 2      5      -1          
(2, 0) 1      3      -1          


In [69]:
sequence, active_k_tuples = construct_active_k_tuples_and_sequence(string_to_symbol_list("012012")[0], 2)
print("active_k_tuples before")
KTupleInfo.print_dict(active_k_tuples)
priority_queue = construct_priority_queue(active_k_tuples)
print("active_k_tuples after")
KTupleInfo.print_dict(active_k_tuples)
print("priority queue")
pprint(priority_queue)

active_k_tuples before
Key    Count  Last   PosInQueue  
--------------------------------
(0, 1) 2      4      -1          
(1, 2) 2      5      -1          
(2, 0) 1      3      -1          
active_k_tuples after
Key    Count  Last   PosInQueue  
--------------------------------
(0, 1) 2      4      0           
(1, 2) 2      5      1           
(2, 0) 1      3      0           
priority queue
{1: [(2, 0)], 2: [(0, 1), (1, 2)]}
